In [1]:
!pip install librosa pydub torch transformers pytube
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00


In [2]:
from pytube import YouTube
import os,shutil

import audioread
from IPython.display import Audio
import librosa
from pydub import AudioSegment, silence

import torch
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration, Wav2Vec2Processor, HubertForCTC

##Загрузка pre-trained Hubert model (используется Connectionist Temporal Classification/CTC loss) с HuggingFace

**Модель предназначена для английского языка**


In [3]:
model_name = "facebook/hubert-large-ls960-ft"
tokenizer = Wav2Vec2Processor.from_pretrained(model_name)
model = HubertForCTC.from_pretrained(model_name)

##Загрузка аудио

In [4]:
def download_audio(url=None):
    for file in os.listdir():
        if file.endswith("mp4"):
            os.remove(file)
    if url!=None:
        yt=YouTube(url)
        print(yt.title)
        stream=list(yt.streams.filter(only_audio=True, file_extension='mp4'))
        stream[0].download() # stream has all .mp4 audios
    else:
        print("Invalid url,can't download")

In [5]:
url="https://www.youtube.com/watch?v=CgxDdsiXf8U&ab_channel=paulczar"
download_audio(url)

George Carlin: Hand Washing


##mp4 to wav (sample rate 16000)

In [6]:
def convert_to_wav(input_filename):
    for file in os.listdir():
        if file.endswith("wav"):
            os.remove(file)
    ext=input_filename[-3:]
    output_filename=filename[:-3]+"wav"
    if ext=="mp3":
        sound = AudioSegment.from_mp3(input_filename)
    else:
        sound = AudioSegment.from_file(input_filename,format=ext)
    sound = sound.set_frame_rate(16000)
    sound.export(output_filename,format="wav")
    os.remove(input_filename)
    return output_filename

In [7]:
filename=""
for file in os.listdir():
    if file.endswith("mp4"):
        filename=file
print("old filename ",filename)
filename = convert_to_wav(filename)
print("new filename ", filename)

old filename  George Carlin Hand Washing.mp4
new filename  George Carlin Hand Washing.wav


##Временная папка для чанков

In [8]:
tmp_dir="audio_chunks"

shutil.rmtree(f"{tmp_dir}/",ignore_errors=True)
os.makedirs(tmp_dir)

##ASR

In [9]:
audio = AudioSegment.from_file(filename)
dBs=audio.dBFS # get decibels 
silence_list=silence.detect_silence(audio,min_silence_len=750,silence_thresh=dBs-14)
silence_list

[[3928, 5038], [33642, 34547]]

In [27]:
#test = audio[55175:59690]
#path = "/content/test_3.wav"
#test.export(path) #Exports to a mp3 file in the current path.
    
#Audio(path, autoplay=False)

In [11]:
def create_chunk(audio,silence_list,threshold=14,max_interval=20000):

    audio_length = int(audio.duration_seconds)*1000 ## in (ms)
    non_silent_chunk=[]
    if len(silence_list)>0:
        ## for 1st chunk 
        if silence_list[0][0]!=0:
            nss=0 # non-silence chunk start
            nse=silence_list[0][0] # non-silence chunk end
            non_silent_chunk.append([nss,nse])
        for idx in range(1,len(silence_list)):
            nss=silence_list[idx-1][1]  # end of previous silence-chunk
            nse=silence_list[idx][0]  # start of current silence-chunk
            non_silent_chunk.append([nss,nse])

        # after last silence chunk 
        if silence_list[-1][1]!=audio_length:
            nss=silence_list[-1][1]
            nse=audio_length
            non_silent_chunk.append([nss,nse])
    else:
        non_silent_chunk.append([0,audio_length])

    print("non_silent_chunk : ",non_silent_chunk)
    new_non_silent_chunk = [] # larger non-silence chunk to smaller sub-chunks
    # sliding window to get audio without silence
    for idx in range(len(non_silent_chunk)):
        start=non_silent_chunk[idx][0]
        end=non_silent_chunk[idx][1]
        interval = end-start
        if interval>max_interval:
            s=start
            while interval>max_interval:
                e=s+max_interval+threshold
                interval=interval-max_interval
                new_non_silent_chunk.append([s,e])
                s=e-threshold
                start=s 
            if interval<=max_interval:
                end=start+interval
                new_non_silent_chunk.append([start,end])
        else:
            new_non_silent_chunk.append([start,end])

    return new_non_silent_chunk

In [12]:
# test = audio[75690:95704]
# path = "/content/test_4.wav"
# test.export(path) #Exports to a mp3 file in the current path.
# Audio(path, autoplay=False)

In [13]:
def transcribe_audio(path,model,tokenizer,audio,start,end,overlap=15):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    try:
        with torch.no_grad():
            new_audio = audio[start:end] 
            new_audio.export(path)  
            print(path)
            input_audio,sr=librosa.load(path,sr=16000)
            input_values = tokenizer(input_audio,return_tensors="pt").to(device).input_values
            logits = model.to(device)(input_values).logits
            prediction = torch.argmax(logits, dim=-1)
            transcription = tokenizer.batch_decode(prediction)[0].lower()
            transcription_start=transcription[:overlap]
            transcription_end=transcription[-overlap:]
            return transcription,transcription_start,transcription_end
    except audioread.NoBackendError:
        print("start value of chunk > end value of chunk")
        exit()


In [14]:
# max_interval - if audio chunk size > max_interval we breake it into chunk
# threshold - gap between 2 chunks ,it uses soft boundary during transition of chunks
new_non_silent_chunk = create_chunk(audio,silence_list,threshold=14,max_interval=20000)
print("new_non_silent_chunk",new_non_silent_chunk)

non_silent_chunk :  [[0, 3928], [5038, 33642], [34547, 84000]]
new_non_silent_chunk [[0, 3928], [5038, 25052], [25038, 33642], [34547, 54561], [54547, 74561], [74547, 84000]]


In [15]:
overlap=20 ## change this value according to need
overlapping_transcription=[]
transcription= ""
root_path="audio_chunks"
for idx in range(len(new_non_silent_chunk)):
    start=new_non_silent_chunk[idx][0]
    end=new_non_silent_chunk[idx][1]
    path=f"{root_path}/chunk_{idx}.wav"
    orginal_trans,trans_start,trans_end=transcribe_audio(path,model,tokenizer,audio,start,end,overlap)
    transcription+=orginal_trans+" "
    overlapping_transcription.append([trans_start,trans_end])

audio_chunks/chunk_0.wav


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


audio_chunks/chunk_1.wav


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


audio_chunks/chunk_2.wav


...

audio_chunks/chunk_3.wav


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


audio_chunks/chunk_4.wav


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


audio_chunks/chunk_5.wav


In [16]:
transcription

"and speaking of my colin want you to know i don't automatically wash my hands every time i go to the bathroom okay can you deal with that sometimes i do sometimes i don't you know when i wash my hands when i shed on them that's the only time that's yo and you know how often that happens tops tops two three times a week tops topmaybe a little more frequently over the holidays you know what i mean tell you something else my well scrubbed friends you don't always need a shower every day did you know that it's overkill unless you work out or work outdoors or for some reason come in intimate contact with huge amounts of filth and garbage every day you don't always need a shower all you really need to do is to wash the four key area armpits ashol crotch an te ot tat armpits ashalt crotch an tee in fact you can save yourself a whole lot of time if you simply use the same brush on all four aras pai  a mesa iyo a plug "

In [17]:
overlapping_transcription

[['and speaking of my c', 'speaking of my colin'],
 ['want you to know i d', 'w often that happens'],
 ['tops tops two three ', 'you know what i mean'],
 ['tell you something e', 'sh the four key area'],
 ['armpits ashol crotch', 'ush on all four aras'],
 ['pai  a mesa iyo a pl', 'i  a mesa iyo a plug']]

In [18]:
model_name="flexudy/t5-small-wav2vec2-grammar-fixer"
t5_tokenizer=T5Tokenizer.from_pretrained(model_name)
t5_model=T5ForConditionalGeneration.from_pretrained(model_name)

In [19]:
def add_punctuation(t5_model,t5_tokenizer,transcription):
    input_text="fix:{"+transcription+"}</s>"
    input_ids=t5_tokenizer.encode(input_text,return_tensors="pt",max_length=10000,truncation=True,add_special_tokens=True)
    outputs=t5_model.generate(input_ids=input_ids,max_length=256,num_beams=4,repetition_penalty=1.0,
                              length_penalty=1.0,early_stopping=True)
    return t5_tokenizer.decode(outputs[0],skip_special_tokens=True,clean_up_tokenization_spaces=True)
    

In [20]:
transcription_punctuated = add_punctuation(t5_model, t5_tokenizer, transcription)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [21]:
transcription_punctuated

"and speaking of my colin want you to know i don't automatically wash my hands every time i go to the bathroom. Okay, can you deal with that? Sometimes i do sometimes i don't know when i wash my hands when i shed on them. That's the only time that's yo, and you know how often that happens tops two three times a week tops topmaybe a little more frequently over the holidays. You know what i mean, tell you something else. My well scrubbed friends you don't always need a shower every day. You don't always need a shower. All you really need to do is to wash the four key area armpits. Ashol crochet an te ot-tat armpits ashalt crochet an tee. In fact, you can save yourself a whole lot of time if you simply use the same brush on all four araspai (a mesa iyo a plug)."